# Final Notebook
# Created by: [Aaron Ooi, Viet Nguyen, Trevor Mathisen]

### Imports and setup

In [1]:
final_models = [] # Add your final models here as a tuple of ('name', model)) for Part 3
%load_ext cuml.accel

cuML: Could not enable managed memory.
cuML: Installed accelerator for sklearn.
cuML: Successfully initialized accelerator.


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import tensorflow as tf

from numpy import ndarray, argsort, set_printoptions, argmax, isnan, nan, mean, random

from pandas import set_option, read_csv, DataFrame, concat, Series
from pandas.plotting import scatter_matrix

from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, Normalizer
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, BaggingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, f1_score, roc_auc_score, mean_squared_error, r2_score
from sklearn.metrics import auc as auc_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

#from cuml.svm import SVC
# Timing



2025-05-07 16:03:09.040710: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-07 16:03:09.047767: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746658989.054605   61608 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746658989.056619   61608 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746658989.065950   61608 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

### Model 4: Support Vector Machines

In [3]:
from sklearn.metrics import auc as auc_score
from perform_kickstarer_eda import X_train, X_test, y_train, y_test
# TODO: REMOVE, ITS ALREADY ABOVEW

ks_data shape: (20632, 68)
Categorical columns: ['country', 'currency', 'deadline', 'state_changed_at', 'created_at', 'launched_at', 'category', 'deadline_weekday', 'state_changed_at_weekday', 'created_at_weekday', 'launched_at_weekday', 'launch_to_deadline', 'launch_to_state_change']
Converted boolean column: staff_pick
Dropped 'state_changed_at_weekday' column
Dropping original column: category
Encoded column: category → 25 features
Dropping original column: deadline_weekday
Encoded column: deadline_weekday → 7 features
Dropping original column: created_at_weekday
Encoded column: created_at_weekday → 7 features
Dropping original column: launched_at_weekday
Encoded column: launched_at_weekday → 7 features
Total categorical columns after encoding: 54


In [ ]:
from cuml.svm import SVC as cuSVC
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, roc_curve
from sklearn.metrics import auc as auc_score
import numpy as np
import matplotlib.pyplot as plt
import cudf
from cuml.metrics import accuracy_score
import cupy as cp

def evaluate_svc_models_rapids(models, X, y):
    results = []
    names = []

    # Convert to GPU if needed (assuming X and y are numpy arrays)
    X_gpu = cudf.DataFrame(X) if not isinstance(X, cudf.DataFrame) else X
    y_gpu = cudf.Series(y) if not isinstance(y, cudf.Series) else y

    for name, _ in models:
        print(f"Tuning hyperparameters for: {name}")
        kfold = KFold(n_splits=10, shuffle=True, random_state=42)

        if name == 'SVC_RBF':
            param_grid = {
                'C': [0.1, 1, 10],
                'gamma': ['scale', 'auto', 0.01, 0.1]
            }
            # cuML SVC doesn't use 'probability' parameter like sklearn
            model = cuSVC(kernel='rbf')
        elif name == 'SVC_POLY':
            param_grid = {
                'C': [0.1, 1, 10],
                'degree': [2, 3, 4],
                'gamma': ['scale', 'auto']
            }
            model = cuSVC(kernel='poly')
        elif name == 'SVC_LINEAR':
            param_grid = {
                'C': [0.1, 1, 10],
                'tol': [1e-4, 1e-3]
            }
            model = cuSVC(kernel='linear')
        elif name == 'SVC_SIGMOID':
            param_grid = {
                'C': [0.1, 1, 10],
                'gamma': ['scale', 'auto']
            }
            model = cuSVC(kernel='sigmoid')
        else:
            continue

        # Manual grid search for cuML SVC (since GridSearchCV isn't directly compatible)
        best_score = 0
        best_params = {}
        best_model = None

        # Generate parameter combinations
        param_combinations = []
        for param_name, param_values in param_grid.items():
            if not param_combinations:
                param_combinations = [{param_name: value} for value in param_values]
            else:
                new_combinations = []
                for combo in param_combinations:
                    for value in param_values:
                        new_combo = combo.copy()
                        new_combo[param_name] = value
                        new_combinations.append(new_combo)
                param_combinations = new_combinations

        # Perform manual grid search with cross-validation
        for params in param_combinations:
            current_model = cuSVC(kernel=model.kernel, **params)
            fold_scores = []

            for train_idx, val_idx in kfold.split(X):
                # Extract folds (using numpy indices works with cuDF)
                X_train_fold = X_gpu.iloc[train_idx]
                y_train_fold = y_gpu.iloc[train_idx]
                X_val_fold = X_gpu.iloc[val_idx]
                y_val_fold = y_gpu.iloc[val_idx]

                # Train and evaluate
                current_model.fit(X_train_fold, y_train_fold)
                y_pred = current_model.predict(X_val_fold)
                score = accuracy_score(y_val_fold, y_pred)
                fold_scores.append(score)

            # Calculate mean score across folds
            mean_score = sum(fold_scores) / len(fold_scores)

            # Update best model if this one is better
            if mean_score > best_score:
                best_score = mean_score
                best_params = params
                best_model = current_model

        # Train best model on full dataset
        best_model = cuSVC(kernel=model.kernel, **best_params)
        best_model.fit(X_gpu, y_gpu)

        print(f"Best params for {name}: {best_params}")
        print(f"Best CV accuracy: {best_score:.4f}")

        # For metrics, convert predictions back to numpy for sklearn compatibility
        # Manual cross-validation for F1 score
        y_pred_list = []
        y_true_list = []

        # Temporary workaround for ROC since cuML SVC doesn't have predict_proba
        # We'll use decision_function instead which gives distance from hyperplane
        decision_scores_list = []

        for train_idx, val_idx in kfold.split(X):
            X_train_fold = X_gpu.iloc[train_idx]
            y_train_fold = y_gpu.iloc[train_idx]
            X_val_fold = X_gpu.iloc[val_idx]
            y_val_fold = y_gpu.iloc[val_idx]

            fold_model = cuSVC(kernel=model.kernel, **best_params)
            fold_model.fit(X_train_fold, y_train_fold)

            y_pred = fold_model.predict(X_val_fold)

            # Use decision_function instead of predict_proba for ROC curve
            decision_scores = fold_model.decision_function(X_val_fold)

            # Handle different return types (cuDF Series/DataFrame or numpy array)
            if hasattr(y_pred, 'to_numpy'):
                y_pred_list.extend(y_pred.to_numpy())
            else:
                y_pred_list.extend(y_pred)

            if hasattr(y_val_fold, 'to_numpy'):
                y_true_list.extend(y_val_fold.to_numpy())
            else:
                y_true_list.extend(y_val_fold)

            if hasattr(decision_scores, 'to_numpy'):
                decision_scores_list.extend(decision_scores.to_numpy())
            else:
                decision_scores_list.extend(decision_scores)

        # Calculate F1 score
        f1 = f1_score(np.array(y_true_list), np.array(y_pred_list))
        print(f"F1 Score for {name}: {f1:.4f}")

        # ROC Curve using decision function scores instead of probabilities
        fpr, tpr, _ = roc_curve(np.array(y_true_list), np.array(decision_scores_list))
        roc_auc = auc_score(fpr, tpr)
        plt.plot(fpr, tpr, label=f"{name} (AUC = {roc_auc:.2f})")

        # Save results for boxplot
        cv_results = np.array(fold_scores)
        results.append(cv_results)
        names.append(name)

        print(f"F1 Score: {f1:.4f} | AUC: {roc_auc:.4f}")

    # Plot results
    fig = plt.figure()
    fig.suptitle('Tuned cuML SVC Model Comparison')
    ax = fig.add_subplot(111)
    plt.boxplot(results, labels=names)
    ax.set_ylabel('Accuracy')
    plt.show()

    return results, names

# Define models
models = [('SVC_RBF', None), ('SVC_POLY', None), ('SVC_LINEAR', None), ('SVC_SIGMOID', None)]

# Run evaluation
evaluate_svc_models_rapids(models, X_train, y_train)

Tuning hyperparameters for: SVC_RBF
Best params for SVC_RBF: {'C': 10, 'gamma': 0.01}
Best CV accuracy: 0.8031
F1 Score for SVC_RBF: 0.6100
F1 Score: 0.6100 | AUC: 0.8562
Tuning hyperparameters for: SVC_POLY
Best params for SVC_POLY: {'C': 10, 'degree': 3, 'gamma': 'scale'}
Best CV accuracy: 0.8010
F1 Score for SVC_POLY: 0.6126
F1 Score: 0.6126 | AUC: 0.8553
Tuning hyperparameters for: SVC_LINEAR
SVC with the linear kernel can be much faster using the specialized solver provided by LinearSVC. Consider switching to LinearSVC if tranining takes too long.
